In [2]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pyautogui as p
import time
import requests

### Grab player data using fetchplayerstats class

In [22]:
from NBAData.fetchPlayersStats import FetchPlayersStats

nba = FetchPlayersStats()
data = nba.getCompleteStats(season='2020-21', season_type='Playoffs', sleep_time=1, max_workers=5)
data

Fetching player stats...
Basic data completed for 2020-21
Fetching advanced player stats...
Fetched advanced stats for 1 of 85 games
Fetched advanced stats for 2 of 85 games
Fetched advanced stats for 3 of 85 games
Fetched advanced stats for 4 of 85 games
Fetched advanced stats for 5 of 85 games
Fetched advanced stats for 6 of 85 games
Fetched advanced stats for 7 of 85 games
Fetched advanced stats for 8 of 85 games
Fetched advanced stats for 9 of 85 games
Fetched advanced stats for 10 of 85 games
Fetched advanced stats for 11 of 85 games
Fetched advanced stats for 12 of 85 games
Fetched advanced stats for 13 of 85 games
Fetched advanced stats for 14 of 85 games
Fetched advanced stats for 15 of 85 games
Fetched advanced stats for 16 of 85 games
Fetched advanced stats for 17 of 85 games
Fetched advanced stats for 18 of 85 games
Fetched advanced stats for 19 of 85 games
Fetched advanced stats for 20 of 85 games
Fetched advanced stats for 21 of 85 games
Fetched advanced stats for 22 of 85

c:\Users\alexg\OneDrive\Documents\Prize-Picks-Prop-Predictor\NBAData\fetchPlayersStats.py:144: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat(team_data, ignore_index=True)


Adding opponent statistics...
Adding offensive ratings...
Adding pace statistics...
Merging player and team data...
Complete stats processing finished!


,PLAYER_NAME,PLAYER_ID,MATCHUP,TEAM_ABBREVIATION,TEAM_ID,OPP_ABBREVIATION,HOME_GAME,GAME_ID,GAME_DATE,WL,MIN,PTS,AST,REB,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,STL,BLK,TOV,PF,PLUS_MINUS,FANTASY_PTS,PointsPerShot,eFG,START_POSITION,OFF_RATING,DEF_RATING,NET_RATING,OREB_PCT,DREB_PCT,REB_PCT,AST_PCT,AST_TOV,USG_PCT,TS_PCT,E_PACE,PACE,PIE,PACE_PER40,TEAM_GAME_DATE,TEAM_MIN,TEAM_FGM,TEAM_FGA,TEAM_FG_PCT,TEAM_FG3M,TEAM_FG3A,TEAM_FG3_PCT,TEAM_FTM,TEAM_FTA,TEAM_FT_PCT,TEAM_OREB,TEAM_DREB,TEAM_REB,TEAM_AST,TEAM_STL,TEAM_BLK,TEAM_TOV,TEAM_PF,TEAM_PTS,OPP_DEF_RATING,OPP_STL,OPP_BLK,OPP_REB,OPP_FG_PCT,OPP_TEAM_ID,TEAM_OFF_RATING,TEAM_PACE,GAME_PACE,OPP_PACE
0,Pat Connaughton,1626192,MIL vs. MIA,MIL,1610612749,MIA,1,0042000121,2021-05-22,W,10,2,1,4,1,2,0.500,0,1,0.000,0,0,NaN,2,2,0,0,1,1,-8,7.3,1.000,1.000,,63.6,100.0,-36.4,0.167,0.154,0.160,0.200,1.00,0.120,0.500,98.91,104.21,0.085,86.84,"MAY 22, 2021",265,42,96,0.438,5,31,0.161,20,33,0.606,15,49,64,19,11,1,17,18,109,96.871667,8.0,3.0,51.0,0.363636,1.610613e+09,96.871667,112.52,112.44,112.36
1,Kristaps Porziņģis,204001,DAL @ LAC,DAL,1610612742,LAC,0,0042000171,2021-05-22,W,36,14,1,4,4,13,0.308,1,5,0.200,5,6,0.833,0,4,0,1,1,2,13,22.3,0.895,0.895,F,135.9,115.6,20.3,0.000,0.105,0.061,0.040,1.00,0.233,0.448,87.16,85.85,0.047,71.54,"MAY 22, 2021",240,38,76,0.500,17,36,0.472,20,26,0.769,10,32,42,22,2,3,12,21,113,126.341682,7.0,7.0,39.0,0.440476,1.610613e+09,126.341682,89.44,88.00,86.56
2,Payton Pritchard,1630202,BOS @ BKN,BOS,1610612738,BKN,0,0042000111,2021-05-22,L,7,0,1,0,0,1,0.000,0,1,0.000,0,0,NaN,0,0,0,0,0,1,0,1.5,0.000,0.000,,130.8,121.4,9.3,0.000,0.000,0.000,0.200,0.00,0.067,0.000,91.67,97.44,-0.026,81.20,"MAY 22, 2021",240,31,84,0.369,11,30,0.367,20,26,0.769,11,29,40,19,6,12,12,18,93,96.433015,7.0,7.0,50.0,0.416667,1.610613e+09,96.433015,96.44,95.10,93.76
3,Tremont Waters,1629682,BOS @ BKN,BOS,1610612738,BKN,0,0042000111,2021-05-22,L,1,0,0,0,0,1,0.000,0,1,0.000,0,0,NaN,0,0,0,0,0,0,0,0.0,0.000,0.000,,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.00,1.000,0.000,48.00,144.00,-1.000,120.00,"MAY 22, 2021",240,31,84,0.369,11,30,0.367,20,26,0.769,11,29,40,19,6,12,12,18,93,96.433015,7.0,7.0,50.0,0.416667,1.610613e+09,96.433015,96.44,95.10,93.76
4,Goran Dragic,201609,MIA @ MIL,MIA,1610612748,MIL,0,0042000121,2021-05-22,L,35,25,1,2,10,17,0.588,5,10,0.500,0,0,NaN,1,1,1,0,4,3,0,27.9,1.471,1.471,,97.1,95.8,1.4,0.024,0.024,0.024,0.077,0.25,0.266,0.735,98.63,97.66,0.144,81.39,"MAY 22, 2021",265,36,99,0.364,20,50,0.400,15,19,0.789,12,39,51,22,8,3,17,23,107,95.229619,11.0,1.0,64.0,0.437500,1.610613e+09,95.229619,112.36,112.44,112.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1859,Jae Crowder,203109,PHX @ MIL,PHX,1610612756,MIL,0,0042000406,2021-07-20,L,41,15,0,13,4,11,0.364,2,9,0.222,5,5,1.000,2,11,4,1,2,3,4,43.6,1.136,1.136,F,100.0,94.3,5.7,0.053,0.282,0.169,0.000,0.00,0.165,0.568,102.34,102.39,0.140,85.33,"JUL 20, 2021",240,38,86,0.442,6,25,0.240,16,19,0.842,6,31,37,14,11,4,14,21,98,95.740524,10.0,6.0,53.0,0.451220,1.610613e+09,95.740524,102.36,102.06,101.76
1860,Chris Paul,101108,PHX @ MIL,PHX,1610612756,MIL,0,0042000406,2021-07-20,L,39,26,5,2,11,19,0.579,1,2,0.500,3,4,0.750,1,1,1,0,3,5,-8,35.9,1.252,1.252,G,91.7,101.2,-9.5,0.026,0.027,0.026,0.263,1.67,0.258,0.626,104.06,102.81,0.123,85.68,"JUL 20, 2021",240,38,86,0.442,6,25,0.240,16,19,0.842,6,31,37,14,11,4,14,21,98,95.740524,10.0,6.0,53.0,0.451220,1.610613e+09,95.740524,102.36,102.06,101.76
1861,Jrue Holiday,201950,MIL vs. PHX,MIL,1610612749,PHX,1,0042000406,2021-07-20,W,46,12,11,9,4,19,0.211,2,7,0.286,2,2,1.000,1,8,4,0,3,1,12,48.3,0.604,0.604,G,106.1,93.9,12.1,0.023,0.167,0.098,0.333,3.67,0.202,0.302,103.25,102.67,0.097,85.56,"JUL 20, 2021",240,37,82,0.451,6,27,0.222,2

In [23]:
data.to_csv('PLAYOFF_DATA/PLAYOFFS_21.csv')

In [6]:
pd.set_option('display.max_columns', None)
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25_FEATURES.csv')
data = data.drop(columns=['Unnamed: 0'])
data['OPP_TEAM_ID'] = data['OPP_TEAM_ID'].fillna(0).astype(int)
data.head()

,PLAYER_NAME,PLAYER_ID,MATCHUP,TEAM_ABBREVIATION,TEAM_ID,OPP_ABBREVIATION,HOME_GAME,GAME_ID,GAME_DATE,WL,MIN,PTS,AST,REB,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,STL,BLK,TOV,PF,PLUS_MINUS,FANTASY_PTS,PointsPerShot,eFG,START_POSITION,OFF_RATING,DEF_RATING,NET_RATING,OREB_PCT,DREB_PCT,REB_PCT,AST_PCT,AST_TOV,USG_PCT,TS_PCT,E_PACE,PACE,PIE,PACE_PER40,TEAM_GAME_DATE,TEAM_MIN,TEAM_FGM,TEAM_FGA,TEAM_FG_PCT,TEAM_FG3M,TEAM_FG3A,TEAM_FG3_PCT,TEAM_FTM,TEAM_FTA,TEAM_FT_PCT,TEAM_OREB,TEAM_DREB,TEAM_REB,TEAM_AST,TEAM_STL,TEAM_BLK,TEAM_TOV,TEAM_PF,TEAM_PTS,OPP_DEF_RATING,OPP_STL,OPP_BLK,OPP_REB,OPP_FG_PCT,OPP_TEAM_ID,TEAM_OFF_RATING,TEAM_PACE,GAME_PACE,OPP_PACE,STARTING,USG_PCT_LAST_3,USG_PCT_LAST_5,USG_PCT_LAST_7,PTS_LAST_3,PTS_LAST_5,PTS_LAST_7,STD_PTS_LAST_3,STD_PTS_LAST_5,STD_PTS_LAST_7,PLAYER_HOME_AVG_PTS,PLAYER_AWAY_AVG_PTS,GUARD,FORWARD,CENTER
0,LeBron James,2544,LAL vs. MIN,LAL,1610612747,MIN,1,42400161,2025-04-19,L,36,19,3,5,8,18,0.444,1,5,0.200,2,2,1.000,1,4,2,3,4,0,-22,40.5,1.006,1.006,F,108.7,142.6,-34.0,0.029,0.138,0.078,0.158,0.75,0.278,0.503,90.65,90.70,0.099,75.59,"APR 19, 2025",240,33,83,0.398,15,41,0.366,14,16,0.875,13,25,38,15,5,4,13,10,95,105.508663,8.0,4.0,44.0,0.511628,1610612750,105.508663,90.04,89.72,89.40,1,0.28,0.28,0.28,19.00,19.00,19.00,NaN,NaN,NaN,19.00,NaN,0,1,0
1,LeBron James,2544,LAL vs. MIN,LAL,1610612747,MIN,1,42400162,2025-04-22,W,40,21,7,11,8,19,0.421,1,5,0.200,4,4,1.000,3,8,1,1,2,4,9,48.7,1.012,1.012,F,111.4,98.6,12.9,0.094,0.195,0.151,0.368,3.50,0.284,0.506,88.21,84.81,0.224,70.68,"APR 22, 2025",240,34,75,0.453,6,29,0.207,20,20,1.000,9,32,41,23,4,6,12,24,94,108.294931,6.0,2.0,34.0,0.379747,1610612750,108.294931,86.80,89.40,92.00,1,0.28,0.28,0.28,20.00,20.00,20.00,1.41,1.41,1.41,20.00,NaN,0,1,0
2,LeBron James,2544,LAL @ MIN,LAL,1610612747,MIN,0,42400163,2025-04-25,L,41,38,4,10,13,21,0.619,5,9,0.556,7,10,0.700,3,7,2,2,1,0,-10,67.0,1.496,1.496,F,109.9,117.9,-8.0,0.077,0.179,0.128,0.211,4.00,0.286,0.748,98.35,96.23,0.266,80.19,"APR 25, 2025",240,37,78,0.474,17,40,0.425,13,20,0.650,7,31,38,21,5,7,16,23,104,108.559499,10.0,5.0,38.0,0.483516,1610612750,108.559499,95.80,97.52,99.24,1,0.28,0.28,0.28,26.00,26.00,26.00,10.44,10.44,10.44,NaN,38.0,0,1,0
3,LeBron James,2544,LAL @ MIN,LAL,1610612747,MIN,0,42400164,2025-04-27,L,46,27,8,12,5,9,0.556,2,4,0.500,15,18,0.833,3,9,3,3,3,2,-4,68.4,1.596,1.596,F,119.1,125.0,-5.9,0.070,0.158,0.120,0.276,2.67,0.186,0.798,92.80,91.88,0.207,76.57,"APR 27, 2025",240,36,80,0.450,19,47,0.404,22,25,0.880,11,30,41,23,6,7,10,23,113,125.555556,8.0,2.0,49.0,0.422222,1610612750,125.555556,90.00,92.76,95.52,1,0.25,0.26,0.26,28.67,26.25,26.25,8.62,8.54,8.54,NaN,32.5,0,1,0
4,LeBron James,2544,LAL vs. MIN,LAL,1610612747,MIN,1,42400165,2025-04-30,L,40,22,6,7,9,21,0.429,1,5,0.200,3,6,0.500,2,5,2,0,3,2,-14,42.4,0.931,0.931,F,93.8,112.7,-18.9,0.049,0.111,0.081,0.375,2.00,0.292,0.465,95.53,94.34,0.132,78.62,"APR 30, 2025",240,33,78,0.423,11,37,0.297,19,25,0.760,8,29,37,24,5,1,15,25,96,100.000000,8.0,5.0,54.0,0.404494,1610612750,100.000000,96.00,96.26,96.52,1,0.25,0.27,0.27,29.00,25.40,25.40,8.19,7.64,7.64,20.67,NaN,0,1,0


In [25]:
from NBAData.features import add_PTS_features
from NBAData.TBD import *

data['STARTING'] = data['START_POSITION'].apply(lambda x: 1 if x in ['G','F','C'] else 0)
data = add_PTS_features(data)
data = assign_position(data)
data

Gathered data for PLAYER_ID 2544...
Gathered data for PLAYER_ID 2546...
Gathered data for PLAYER_ID 2730...
Gathered data for PLAYER_ID 2738...
Gathered data for PLAYER_ID 2772...
Gathered data for PLAYER_ID 101108...
Gathered data for PLAYER_ID 101141...
Gathered data for PLAYER_ID 101150...
Gathered data for PLAYER_ID 200765...
Gathered data for PLAYER_ID 200782...
Gathered data for PLAYER_ID 200794...
Gathered data for PLAYER_ID 201142...
Gathered data for PLAYER_ID 201144...
Gathered data for PLAYER_ID 201145...
Gathered data for PLAYER_ID 201162...
Gathered data for PLAYER_ID 201188...
Gathered data for PLAYER_ID 201229...
Gathered data for PLAYER_ID 201565...
Gathered data for PLAYER_ID 201566...
Gathered data for PLAYER_ID 201568...
Gathered data for PLAYER_ID 201572...
Gathered data for PLAYER_ID 201577...
Gathered data for PLAYER_ID 201580...
Gathered data for PLAYER_ID 201586...
Gathered data for PLAYER_ID 201587...
Gathered data for PLAYER_ID 201588...
Gathered data for PLAY

,PLAYER_NAME,PLAYER_ID,MATCHUP,TEAM_ABBREVIATION,TEAM_ID,OPP_ABBREVIATION,HOME_GAME,GAME_ID,GAME_DATE,WL,MIN,PTS,AST,REB,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,STL,BLK,TOV,PF,PLUS_MINUS,FANTASY_PTS,PointsPerShot,eFG,START_POSITION,OFF_RATING,DEF_RATING,NET_RATING,OREB_PCT,DREB_PCT,REB_PCT,AST_PCT,AST_TOV,USG_PCT,TS_PCT,E_PACE,PACE,PIE,PACE_PER40,TEAM_GAME_DATE,TEAM_MIN,TEAM_FGM,TEAM_FGA,TEAM_FG_PCT,TEAM_FG3M,TEAM_FG3A,TEAM_FG3_PCT,TEAM_FTM,TEAM_FTA,TEAM_FT_PCT,TEAM_OREB,TEAM_DREB,TEAM_REB,TEAM_AST,TEAM_STL,TEAM_BLK,TEAM_TOV,TEAM_PF,TEAM_PTS,OPP_DEF_RATING,OPP_STL,OPP_BLK,OPP_REB,OPP_FG_PCT,OPP_TEAM_ID,TEAM_OFF_RATING,TEAM_PACE,GAME_PACE,OPP_PACE,STARTING,USG_PCT_LAST_3,USG_PCT_LAST_5,USG_PCT_LAST_7,PTS_LAST_3,PTS_LAST_5,PTS_LAST_7,STD_PTS_LAST_3,STD_PTS_LAST_5,STD_PTS_LAST_7,PLAYER_HOME_AVG_PTS,PLAYER_AWAY_AVG_PTS,GUARD,FORWARD,CENTER
136,LeBron James,2544,LAL @ PHX,LAL,1610612747,PHX,0,42000151,2021-05-23,L,36,18,10,7,6,13,0.462,3,7,0.429,3,6,0.500,1,6,3,0,5,1,2,45.4,1.151,1.151,F,104.4,98.6,5.8,0.030,0.162,0.100,0.476,2.00,0.263,0.575,89.63,91.84,0.183,76.54,"MAY 23, 2021",240,33,76,0.434,7,26,0.269,17,28,0.607,10,23,33,19,10,5,14,18,90,97.487002,7.0,4.0,47.0,0.465116,1610612756,97.487002,92.32,90.80,89.28,1,0.26,0.26,0.26,18.00,18.00,18.00,NaN,NaN,NaN,NaN,18.00,0,1,0
263,LeBron James,2544,LAL @ PHX,LAL,1610612747,PHX,0,42000152,2021-05-25,W,39,23,9,4,9,16,0.563,4,9,0.444,1,1,1.000,0,4,2,0,1,3,14,46.3,1.399,1.399,F,129.3,112.2,17.2,0.000,0.121,0.058,0.360,9.00,0.193,0.700,94.54,91.95,0.176,76.62,"MAY 25, 2021",240,36,80,0.450,10,33,0.303,27,31,0.871,8,31,39,24,9,6,7,23,109,117.659758,4.0,3.0,31.0,0.464789,1610612756,117.659758,92.64,92.92,93.20,1,0.23,0.23,0.23,20.50,20.50,20.50,3.54,3.54,3.54,NaN,20.50,0,1,0
373,LeBron James,2544,LAL vs. PHX,LAL,1610612747,PHX,1,42000153,2021-05-27,W,38,21,9,6,9,19,0.474,1,5,0.200,2,3,0.667,1,5,1,0,7,0,15,37.7,1.033,1.033,F,106.2,89.9,16.3,0.024,0.125,0.074,0.450,1.29,0.273,0.517,103.57,101.71,0.165,84.76,"MAY 27, 2021",240,38,86,0.442,7,28,0.250,26,30,0.867,15,36,51,19,10,4,20,19,109,104.606526,8.0,6.0,35.0,0.426829,1610612756,104.606526,104.20,101.56,98.92,1,0.24,0.24,0.24,20.67,20.67,20.67,2.52,2.52,2.52,21.00,NaN,0,1,0
577,LeBron James,2544,LAL vs. PHX,LAL,1610612747,PHX,1,42000154,2021-05-30,L,38,25,6,12,10,21,0.476,1,7,0.143,4,7,0.571,2,10,1,0,6,1,6,45.4,1.038,1.038,F,105.3,97.3,8.0,0.057,0.233,0.154,0.353,1.00,0.361,0.519,93.89,95.18,0.167,79.32,"MAY 30, 2021",240,32,81,0.395,13,40,0.325,15,21,0.714,9,41,50,19,3,5,15,18,92,95.594347,12.0,4.0,49.0,0.435294,1610612756,95.594347,96.24,95.74,95.24,1,0.28,0.27,0.27,23.00,21.75,21.75,2.00,2.99,2.99,23.00,NaN,0,1,0
716,LeBron James,2544,LAL @ PHX,LAL,1610612747,PHX,0,42000155,2021-06-01,L,32,24,7,5,9,19,0.474,6,10,0.600,0,0,NaN,1,4,0,0,3,1,-24,37.5,1.263,1.263,F,90.5,128.6,-38.1,0.029,0.121,0.074,0.700,2.33,0.310,0.632,99.78,95.57,0.195,79.64,"JUN 01, 2021",240,29,84,0.345,12,35,0.343,15,21,0.714,10,41,51,14,3,1,16,25,85,85.650947,7.0,4.0,47.0,0.457447,1610612756,85.650947,99.24,101.74,104.24,1,0.31,0.28,0.28,23.33,22.20,22.20,2.08,2.77,2.77,NaN,21.67,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,Facundo Campazzo,1630267,DEN @ POR,DEN,1610612743,POR,0,42000166,2021-06-03,W,23,10,3,2,2,5,0.400,1,4,0.250,5,5,1.000,1,1,1,1,1,5,-2,21.9,1.389,1.389,G,125.0,124.0,1.0,0.045,0.056,0.050,0.167,3.00,0.151,0.694,103.42,101.60,0.062,84.67,"JUN 03, 2021",240,44,86,0.512,15,34,0.441,23,27,0.852,10,29,39,28,8,3,12,18,126,126.151382,7.0,7.0,37.0,0.506329,1610612757,126.151382,99.88,98.28,96.68,1,0.16,0.17,0.15,8.33,9.60,9.33,4.73,3.78,3.44,NaN,11.00,1,0,0
1043,Facundo Campazzo,1630267,DEN @ PHX,DEN,16106

In [26]:
data.to_csv('PLAYOFF_DATA/PLAYOFFS_21_FEATURES.csv')